In [1]:
import nibabel as nb
import numpy as np
import scipy.io
import scipy
from scipy import stats
import math
import pandas as pd
import statsmodels.api as sm

In [2]:
regressor_mean_magnitude_motion_vectors = np.load('D:/place_ROIs_project/Phase5_AnalysisBasedOnPickActivityEachPatch/MotionBrainMaps/MotionMeanMap.npy')
regressor_scene_transitions = np.load('E:\FPP_files_and_codes\FPP_files\scene_transition/regressor_absdif_intensity_pixels/regressor_mean_maxdif_pixel_intensitiy.npy')


<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\2296880036.py:2: SyntaxWarning: invalid escape sequence '\F'
  regressor_scene_transitions = np.load('E:\FPP_files_and_codes\FPP_files\scene_transition/regressor_absdif_intensity_pixels/regressor_mean_maxdif_pixel_intensitiy.npy')


In [5]:
regressor_scene_transitions

array([2.96055170e-03, 1.62519290e-04, 7.23379630e-06, ...,
       0.00000000e+00, 1.44675926e-06, 0.00000000e+00])

In [7]:
delay = 5
sifted_regressor_mean_magnitude_motion_vectors = np.roll(regressor_mean_magnitude_motion_vectors, delay)
sifted_regressor_scene_transitions = np.roll(regressor_scene_transitions, delay)

In [10]:
sifted_regressor_mean_magnitude_motion_vectors.reshape(1, -1).shape

(1, 3655)

In [11]:
left_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_left_Meanfile.mat')['Left_data']
right_data = scipy.io.loadmat('E:/FPP_files_and_codes/FPP_files/Movie_watching_avg_data/notmean_right_Meanfile.mat')['Right_data']

In [13]:
sifted_regressor_mean_magnitude_motion_vectors_2d = sifted_regressor_mean_magnitude_motion_vectors.reshape(1, -1)
sifted_regressor_scene_transitions_2d = sifted_regressor_scene_transitions.reshape(1, -1)
X = np.concatenate((sifted_regressor_mean_magnitude_motion_vectors_2d, sifted_regressor_scene_transitions_2d), axis=0).T

coefficients_regressor_motion_right = np.zeros(32492)
coefficients_regrssor_transition_right = np.zeros(32492)

pvalue_coefficients_regressor_motion_right = np.zeros(32492)
pvalue_coefficients_regrssor_transition_right = np.zeros(32492)


zero = np.zeros(3655)

for (i,vertex_data) in enumerate(right_data):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2'])
    df['y'] = y

    X = sm.add_constant(df[['X1', 'X2']])
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):
        result = model.fit()
        coefficients = result.params

        coefficients_regressor_motion_right[i] = coefficients['X1']
        coefficients_regrssor_transition_right[i] = coefficients['X2']
        
        p_values = result.pvalues
        pvalue_coefficients_regressor_motion_right[i] = p_values['X1']
        pvalue_coefficients_regrssor_transition_right[i] = p_values['X2']

In [15]:
coefficients_regressor_motion_right = np.nan_to_num(coefficients_regressor_motion_right)
coefficients_regrssor_transition_right = np.nan_to_num(coefficients_regrssor_transition_right)

pvalue_coefficients_regressor_motion_right = np.nan_to_num(pvalue_coefficients_regressor_motion_right)
pvalue_coefficients_regrssor_transition_right = np.nan_to_num(pvalue_coefficients_regrssor_transition_right)

coefficients_regressor_motion_right_32 = coefficients_regressor_motion_right.astype(np.float32)
coefficients_regrssor_transition_right_32 = coefficients_regrssor_transition_right.astype(np.float32)

pvalue_coefficients_regressor_motion_right_32 = pvalue_coefficients_regressor_motion_right.astype(np.float32)
pvalue_coefficients_regrssor_transition_right_32 = pvalue_coefficients_regrssor_transition_right.astype(np.float32)

In [16]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = coefficients_regressor_motion_right_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/beta_regressor_motion_regressout_transition.R.func.gii')

In [17]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = coefficients_regrssor_transition_right_32
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/beta_regressor_transition_regressout_motion.R.func.gii')

In [18]:
significance_beta_mean_motion_right = -1* np.log(pvalue_coefficients_regressor_motion_right) * np.sign(coefficients_regressor_motion_right)
significance_beta_transition_right = -1* np.log(pvalue_coefficients_regrssor_transition_right) * np.sign(coefficients_regrssor_transition_right)

significance_beta_mean_motion_right = np.nan_to_num(significance_beta_mean_motion_right, nan=0.0, posinf=0.0, neginf=0.0)
significance_beta_transition_right = np.nan_to_num(significance_beta_transition_right, nan=0.0, posinf=0.0, neginf=0.0)

float32_significance_beta_mean_motion_right = [np.float32(x) for x in significance_beta_mean_motion_right]
float32_significance_beta_transition_right = [np.float32(x) for x in significance_beta_transition_right]

C:\Users\roza\AppData\Local\Temp\ipykernel_111544\1052118391.py:1: RuntimeWarning: divide by zero encountered in log
  significance_beta_mean_motion_right = -1* np.log(pvalue_coefficients_regressor_motion_right) * np.sign(coefficients_regressor_motion_right)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\1052118391.py:1: RuntimeWarning: invalid value encountered in multiply
  significance_beta_mean_motion_right = -1* np.log(pvalue_coefficients_regressor_motion_right) * np.sign(coefficients_regressor_motion_right)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\1052118391.py:2: RuntimeWarning: divide by zero encountered in log
  significance_beta_transition_right = -1* np.log(pvaluecoefficients_regrssor_transition_right) * np.sign(coefficients_regrssor_transition_right)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\1052118391.py:2: RuntimeWarning: invalid value encountered in multiply
  significance_beta_transition_right = -1* np.log(pvaluecoefficients_regrssor_transition_right

In [19]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = float32_significance_beta_mean_motion_right
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/significance_beta_regressor_motion_regressout_transition.R.func.gii')

In [20]:
temp_r_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_r = nb.load(temp_r_path)
temp_r.darrays[0].data = float32_significance_beta_transition_right
temp_r.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/significance_beta_regressor_transition_regressout_motion.R.func.gii')

In [21]:
sifted_regressor_mean_magnitude_motion_vectors_2d = sifted_regressor_mean_magnitude_motion_vectors.reshape(1, -1)
sifted_regressor_scene_transitions_2d = sifted_regressor_scene_transitions.reshape(1, -1)
X = np.concatenate((sifted_regressor_mean_magnitude_motion_vectors_2d, sifted_regressor_scene_transitions_2d), axis=0).T

coefficients_regressor_motion_left = np.zeros(32492)
coefficients_regrssor_transition_left = np.zeros(32492)

pvalue_coefficients_regressor_motion_left = np.zeros(32492)
pvalue_coefficients_regrssor_transition_left = np.zeros(32492)


zero = np.zeros(3655)

for (i,vertex_data) in enumerate(left_data):
    y = vertex_data
    df = pd.DataFrame(X, columns=['X1', 'X2'])
    df['y'] = y

    # Fit GLM model
    X = sm.add_constant(df[['X1', 'X2']])  # Add constant for intercept term
    model = sm.GLM(df['y'], X, family=sm.families.Gaussian())
    if not np.array_equal(vertex_data, zero):
        result = model.fit()
        coefficients = result.params

        coefficients_regressor_motion_left[i] = coefficients['X1']
        coefficients_regrssor_transition_left[i] = coefficients['X2']
        
        p_values = result.pvalues
        pvalue_coefficients_regressor_motion_left[i] = p_values['X1']
        pvalue_coefficients_regrssor_transition_left[i] = p_values['X2']

In [23]:
coefficients_regressor_motion_left = np.nan_to_num(coefficients_regressor_motion_left)
coefficients_regrssor_transition_left = np.nan_to_num(coefficients_regrssor_transition_left)

pvalue_coefficients_regressor_motion_left = np.nan_to_num(pvalue_coefficients_regressor_motion_left)
pvalue_coefficients_regrssor_transition_left = np.nan_to_num(pvalue_coefficients_regrssor_transition_left)

coefficients_regressor_motion_left_32 = coefficients_regressor_motion_left.astype(np.float32)
coefficients_regrssor_transition_left_32 = coefficients_regrssor_transition_left.astype(np.float32)

pvalue_coefficients_regressor_motion_left_32 = pvalue_coefficients_regressor_motion_left.astype(np.float32)
pvalue_coefficients_regrssor_transition_left_32 = pvalue_coefficients_regrssor_transition_left.astype(np.float32)

In [24]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = coefficients_regressor_motion_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/beta_regressor_motion_regressout_transition.L.func.gii')

In [25]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.R.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = coefficients_regrssor_transition_left_32
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/beta_regressor_transition_regressout_motion.L.func.gii')

In [28]:
significance_beta_mean_motion_left = -1* np.log(pvalue_coefficients_regressor_motion_left) * np.sign(coefficients_regressor_motion_left)
significance_beta_transition_left = -1* np.log(pvalue_coefficients_regrssor_transition_left) * np.sign(coefficients_regrssor_transition_left)

significance_beta_mean_motion_left = np.nan_to_num(significance_beta_mean_motion_left, nan=0.0, posinf=0.0, neginf=0.0)
significance_beta_transition_left = np.nan_to_num(significance_beta_transition_left, nan=0.0, posinf=0.0, neginf=0.0)

float32_significance_beta_mean_motion_left = [np.float32(x) for x in significance_beta_mean_motion_left]
float32_significance_beta_transition_left = [np.float32(x) for x in significance_beta_transition_left]

C:\Users\roza\AppData\Local\Temp\ipykernel_111544\3334211662.py:1: RuntimeWarning: divide by zero encountered in log
  significance_beta_mean_motion_left = -1* np.log(pvalue_coefficients_regressor_motion_left) * np.sign(coefficients_regressor_motion_left)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\3334211662.py:1: RuntimeWarning: invalid value encountered in multiply
  significance_beta_mean_motion_left = -1* np.log(pvalue_coefficients_regressor_motion_left) * np.sign(coefficients_regressor_motion_left)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\3334211662.py:2: RuntimeWarning: divide by zero encountered in log
  significance_beta_transition_left = -1* np.log(pvalue_coefficients_regrssor_transition_left) * np.sign(coefficients_regrssor_transition_left)
C:\Users\roza\AppData\Local\Temp\ipykernel_111544\3334211662.py:2: RuntimeWarning: invalid value encountered in multiply
  significance_beta_transition_left = -1* np.log(pvalue_coefficients_regrssor_transition_left) * np.si

In [29]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = float32_significance_beta_mean_motion_left
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/significance_beta_regressor_motion_regressout_transition.L.func.gii')

In [30]:
temp_l_path = 'E:/FPP_files_and_codes/FPP_files/classic_place_patches/ROIs.L.func.gii'
temp_l = nb.load(temp_l_path)
temp_l.darrays[0].data = float32_significance_beta_transition_left
temp_l.to_filename('E:/FPP_files_and_codes/FPP_files/motion_maps/mean_magnitude_motion_regress_out_transition/significance_beta_regressor_transition_regressout_motion.L.func.gii')